In [ ]:
!python --version


In [ ]:
!python -c "import torch; print(torch.cuda.is_available())"


In [ ]:
!pip install transformers datasets torch scikit-learn pandas openpyxl


In [26]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
import numpy as np
import datasets.formatting.formatting
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import StratifiedShuffleSplit
import torch.nn as nn
from torch.nn import functional as F

# -----------------------------------------------------
# ✅ Monkey patch for NumPy 2.0 error in datasets
def fixed_arrow_array_to_numpy(self, pa_array):
    array = pa_array.to_pandas().values
    return np.array(array, copy=True)

datasets.formatting.formatting.NumpyArrowExtractor._arrow_array_to_numpy = fixed_arrow_array_to_numpy
# -----------------------------------------------------

# ✅ Disable wandb
os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Load dataframe with better preprocessing
df = pd.read_excel(r"BanglaBlendCleanedDataWithEnglishTranslation.xlsx")
df = df.dropna()



Using device: cuda


In [27]:
df

,Sentence,Sentence(English Translation),Labels(English),Labels(Bangla),Labels
0,সেখানকার জানালা দিয়ে সমুদ্র দেখা যাইতেছিল,The sea could be seen from the window there,Saint,Sadhu(সাধু ),0
1,আমি কিছু দেখিতে পারিতেছি না,I can't see anything,Saint,Sadhu(সাধু ),0
2,সকলেরই অনাবৃত দেহ সকলের সেই অনাবৃত বক্ষে আরশির...,Everyone's naked body is burning once in the m...,Saint,Sadhu(সাধু ),0
3,মেয়েটি সেদিন ভিক্ষুককে সাহায্য করিয়াছিল,The girl had helped the beggar that day,Saint,Sadhu(সাধু ),0
4,তুমি প্রশংসা কর না কর বৃদ্ধ বসিয়া তোমায় পুরা...,You don't praise the old man and he will tell ...,Saint,Sadhu(সাধু ),0
...,...,...,...,...,...
7345,রন্ধনপ্রণালী দোপেয়াজা জনপ্রিয়,The cuisine Dopeyazza is popular,Common,Cholito(চলিত),1
7346,শেষে রহিম করিমকে বিপদে ফেলল,"At the end, Rahim put Karim in danger",Common,Cholito(চলিত),1
7347,হাফিজ কে তারাই বিপদে ফেলল,They put Hafiz in danger,Common,Cholito(চলিত),1
7348,সালে আলেকজান্ডার কানিংহাম প্রথম হড়প্পা সিলমোহ...,Alexander Cunningham published the first Harap...,Common,Cholito(চলিত),1


In [28]:

# ✅ Keep only needed columns and rename
df = df[["Sentence", "Labels"]].copy()

In [30]:
df = df.rename(columns={"Sentence": "text", "Labels": "label"})
df["label"] = df["label"].astype(int)

# ✅ Basic text preprocessing
def preprocess_text(text):
    """Basic text preprocessing"""
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

df["text"] = df["text"].apply(preprocess_text)

# ✅ Remove empty texts and duplicates
df = df[df["text"].str.len() > 0]
df = df.drop_duplicates(subset=["text"])

print("Columns after clean & rename:", df.columns)
print("Data shape:", df.shape)
print("Label distribution:")
print(df["label"].value_counts())
print(df.head())

# ✅ Load tokenizer and model with built-in regularization
model_name = "sagorsarker/bangla-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use standard model with built-in dropout configuration
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2,
    hidden_dropout_prob=0.3,  # Increase dropout for regularization
    attention_probs_dropout_prob=0.3,  # Attention dropout
    classifier_dropout=0.3  # Classifier dropout
).to(device)

# ✅ Advanced tokenization with dynamic padding
def tokenize(batch):
    return tokenizer(
        batch["text"], 
        padding=False,  # Dynamic padding in data collator
        truncation=True, 
        max_length=256,  # Increased max length for better context
        return_tensors=None
    )

# ✅ Create dataset with stratified split
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# ✅ Stratified train-validation-test split
def create_stratified_splits(dataset, train_size=0.7, val_size=0.15, test_size=0.15, random_state=42):
    labels = [item['label'] for item in dataset]
    
    # First split: train and temp (val + test)
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=(val_size + test_size), random_state=random_state)
    train_idx, temp_idx = next(sss1.split(range(len(labels)), labels))
    
    # Second split: val and test from temp
    temp_labels = [labels[i] for i in temp_idx]
    val_ratio = val_size / (val_size + test_size)
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=(1 - val_ratio), random_state=random_state)
    val_idx_temp, test_idx_temp = next(sss2.split(range(len(temp_idx)), temp_labels))
    
    val_idx = [temp_idx[i] for i in val_idx_temp]
    test_idx = [temp_idx[i] for i in test_idx_temp]
    
    return {
        'train': dataset.select(train_idx),
        'validation': dataset.select(val_idx),
        'test': dataset.select(test_idx)
    }

dataset_splits = create_stratified_splits(dataset)

# ✅ Set torch format
for split in dataset_splits:
    dataset_splits[split].set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

print(f"Train size: {len(dataset_splits['train'])}")
print(f"Validation size: {len(dataset_splits['validation'])}")
print(f"Test size: {len(dataset_splits['test'])}")

# ✅ Data collator for dynamic padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ✅ Enhanced metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



# ✅ Enhanced training arguments with regularization
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=25,  # Reduced epochs to prevent overfitting
    per_device_train_batch_size=32,  # Increased batch size
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=2,  # Effective batch size = 32
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    
    # ✅ Learning rate scheduling
    learning_rate=3e-5,  # Slightly higher learning rate
    lr_scheduler_type="cosine",
    warmup_steps=500,
    
    # ✅ Regularization techniques
    weight_decay=0.01,  # L2 regularization
    adam_epsilon=1e-6,
    max_grad_norm=1.0,  # Gradient clipping
    label_smoothing_factor=0.1,  # Built-in label smoothing
    
    # ✅ Early stopping and model selection
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    
    # ✅ Evaluation and saving
    eval_steps=100,
    save_steps=200,
    save_total_limit=3,
    
    # ✅ Other optimizations
    dataloader_num_workers=2,
    remove_unused_columns=False,
    report_to=[]
)

# ✅ Standard trainer with built-in label smoothing
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_splits["train"],
    eval_dataset=dataset_splits["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator
    # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# ✅ Train with mixed precision for efficiency
print("🚀 Starting training...")
trainer.train()

# ✅ Evaluate on validation set
print("\n📊 Validation Results:")
val_metrics = trainer.evaluate()
for key, value in val_metrics.items():
    print(f"{key}: {value:.4f}")

# ✅ Evaluate on test set
print("\n📊 Test Results:")
test_metrics = trainer.evaluate(dataset_splits["test"])
for key, value in test_metrics.items():
    print(f"test_{key}: {value:.4f}")

# ✅ Calculate overfitting metrics
train_metrics = trainer.evaluate(dataset_splits["train"])
overfitting_accuracy = train_metrics['eval_accuracy'] - test_metrics['eval_accuracy']
overfitting_f1 = train_metrics['eval_f1'] - test_metrics['eval_f1']

print(f"\n🎯 Overfitting Analysis:")
print(f"Training Accuracy: {train_metrics['eval_accuracy']:.4f}")
print(f"Test Accuracy: {test_metrics['eval_accuracy']:.4f}")
print(f"Accuracy Gap (overfitting): {overfitting_accuracy:.4f}")
print(f"F1 Gap (overfitting): {overfitting_f1:.4f}")

# ✅ Save the best model
model.save_pretrained("./final_bangla_bert_model")
tokenizer.save_pretrained("./final_bangla_bert_model")
print("\n✅ Best model saved at ./final_bangla_bert_model")

# ✅ Optional: Model inference example
def predict_text(text, model, tokenizer, device):
    """Function to predict on new text"""
    model.eval()
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=256
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1)
    
    return predicted_class.item(), predictions.cpu().numpy()

# ✅ Example usage
sample_text = "এটি একটি পরীক্ষা বাক্য।"
try:
    pred_class, pred_probs = predict_text(sample_text, model, tokenizer, device)
    print(f"\nSample prediction:")
    print(f"Text: {sample_text}")
    print(f"Predicted class: {pred_class}")
    print(f"Probabilities: {pred_probs}")
except Exception as e:
    print(f"Prediction error: {e}")

Columns after clean & rename: Index(['text', 'label'], dtype='object')
Data shape: (7066, 2)
Label distribution:
label
1    3608
0    3458
Name: count, dtype: int64
                                                text  label
0          সেখানকার জানালা দিয়ে সমুদ্র দেখা যাইতেছিল      0
1                        আমি কিছু দেখিতে পারিতেছি না      0
2  সকলেরই অনাবৃত দেহ সকলের সেই অনাবৃত বক্ষে আরশির...      0
3            মেয়েটি সেদিন ভিক্ষুককে সাহায্য করিয়াছিল      0
4  তুমি প্রশংসা কর না কর বৃদ্ধ বসিয়া তোমায় পুরা...      0


C:\Users\Pust\AppData\Roaming\Python\Python39\site-packages\huggingface_hub\file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sagorsarker/bangla-bert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7066 [00:00<?, ? examples/s]

Train size: 4946
Validation size: 1060
Test size: 1060
🚀 Starting training...


  0%|          | 0/1925 [00:00<?, ?it/s]

...

eval_loss: 0.3247
eval_accuracy: 0.9491
eval_f1: 0.9490
eval_precision: 0.9491
eval_recall: 0.9491
eval_runtime: 7.4967
eval_samples_per_second: 141.3960
eval_steps_per_second: 4.5350
epoch: 24.8387

📊 Test Results:


  0%|          | 0/34 [00:00<?, ?it/s]

test_eval_loss: 0.3210
test_eval_accuracy: 0.9509
test_eval_f1: 0.9509
test_eval_precision: 0.9511
test_eval_recall: 0.9509
test_eval_runtime: 7.6044
test_eval_samples_per_second: 139.3940
test_eval_steps_per_second: 4.4710
test_epoch: 24.8387


  0%|          | 0/155 [00:00<?, ?it/s]


🎯 Overfitting Analysis:
Training Accuracy: 0.9992
Test Accuracy: 0.9509
Accuracy Gap (overfitting): 0.0482
F1 Gap (overfitting): 0.0483

✅ Best model saved at ./final_bangla_bert_model

Sample prediction:
Text: এটি একটি পরীক্ষা বাক্য।
Predicted class: 1
Probabilities: [[0.04099333 0.95900667]]


In [31]:
from sklearn.metrics import classification_report

# ✅ Evaluate on validation set
print("\n📊 Validation Results:")
val_preds_output = trainer.predict(dataset_splits["validation"])
val_labels = val_preds_output.label_ids
val_preds = val_preds_output.predictions.argmax(-1)

val_metrics = trainer.evaluate()
for key, value in val_metrics.items():
    print(f"{key}: {value:.4f}")

# ✅ Classification report on validation
print("\n🔎 Classification Report (Validation):")
print(classification_report(val_labels, val_preds, digits=4))

# ✅ Evaluate on test set
print("\n📊 Test Results:")
test_preds_output = trainer.predict(dataset_splits["test"])
test_labels = test_preds_output.label_ids
test_preds = test_preds_output.predictions.argmax(-1)

test_metrics = trainer.evaluate(dataset_splits["test"])
for key, value in test_metrics.items():
    print(f"test_{key}: {value:.4f}")

# ✅ Classification report on test
print("\n🔎 Classification Report (Test):")
print(classification_report(test_labels, test_preds, digits=4))



📊 Validation Results:


  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

eval_loss: 0.3247
eval_accuracy: 0.9491
eval_f1: 0.9490
eval_precision: 0.9491
eval_recall: 0.9491
eval_runtime: 7.3961
eval_samples_per_second: 143.3190
eval_steps_per_second: 4.5970
epoch: 24.8387

🔎 Classification Report (Validation):
              precision    recall  f1-score   support

           0     0.9549    0.9402    0.9475       518
           1     0.9436    0.9576    0.9505       542

    accuracy                         0.9491      1060
   macro avg     0.9493    0.9489    0.9490      1060
weighted avg     0.9491    0.9491    0.9490      1060


📊 Test Results:


  0%|          | 0/34 [00:00<?, ?it/s]

  0%|          | 0/34 [00:00<?, ?it/s]

test_eval_loss: 0.3210
test_eval_accuracy: 0.9509
test_eval_f1: 0.9509
test_eval_precision: 0.9511
test_eval_recall: 0.9509
test_eval_runtime: 7.5675
test_eval_samples_per_second: 140.0720
test_eval_steps_per_second: 4.4930
test_epoch: 24.8387

🔎 Classification Report (Test):
              precision    recall  f1-score   support

           0     0.9587    0.9403    0.9494       519
           1     0.9437    0.9612    0.9524       541

    accuracy                         0.9509      1060
   macro avg     0.9512    0.9507    0.9509      1060
weighted avg     0.9511    0.9509    0.9509      1060



In [ ]:
df.shape

In [ ]:

import matplotlib.pyplot as plt
import torch
from sklearn.metrics import *
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Get predictions from validation or test set
predictions = trainer.predict(dataset_splits["test"])
y_true = predictions.label_ids
y_pred = predictions.predictions.argmax(-1)

# Plot confusion matrix
cm = confusion_matrix(y_true, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap="Blues", values_format="d")
plt.title("Confusion Matrix (Test Set)")
plt.show()


In [ ]:
import pandas as pd

# Load logs from Trainer
log_history = trainer.state.log_history
log_df = pd.DataFrame(log_history)

# Filter only relevant columns
train_logs = log_df[log_df["loss"].notna()]
eval_logs = log_df[log_df["eval_loss"].notna()]

# Plot Loss
plt.plot(train_logs["step"], train_logs["loss"], label="Train Loss")
# plt.plot(eval_logs["step"], eval_logs["eval_loss"], label="Eval Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.legend()
plt.title("Training Loss")
plt.grid(True)
plt.show()

# Plot Accuracy
if "eval_accuracy" in eval_logs.columns:
    plt.plot(eval_logs["step"], eval_logs["eval_accuracy"], label="Eval Accuracy")
    plt.xlabel("Step")
    plt.ylabel("Accuracy")
    plt.title("Validation Accuracy")
    plt.grid(True)
    plt.legend()
    plt.show()


In [ ]:
import pandas as pd

# Load logs from Trainer
log_history = trainer.state.log_history
log_df = pd.DataFrame(log_history)

# Filter only relevant columns
train_logs = log_df[log_df["loss"].notna()]
eval_logs = log_df[log_df["eval_loss"].notna()]

# Plot Loss
# plt.plot(train_logs["step"], train_logs["loss"], label="Train Loss")
plt.plot(eval_logs["step"], eval_logs["eval_loss"], label="Eval Loss")
plt.xlabel("Step")
plt.ylabel("Loss")
plt.legend()
plt.title("Validation Loss")
plt.grid(True)
plt.show()

# Plot Accuracy
if "eval_accuracy" in eval_logs.columns:
    plt.plot(eval_logs["step"], eval_logs["eval_accuracy"], label="Eval Accuracy")
    plt.xlabel("Step")
    plt.ylabel("Accuracy")
    plt.title("Validation Accuracy")
    plt.grid(True)
    plt.legend()
    plt.show()


In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

# Get softmax probabilities
probs = predictions.predictions
probs_softmax = torch.nn.functional.softmax(torch.tensor(probs), dim=1).numpy()

# ROC Curve (for binary classification only)
fpr, tpr, thresholds = roc_curve(y_true, probs_softmax[:, 1])
auc_score = roc_auc_score(y_true, probs_softmax[:, 1])

plt.plot(fpr, tpr, label=f"AUC = {auc_score:.4f}")
plt.plot([0, 1], [0, 1], linestyle="--", color="gray")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("AUC-ROC Curve")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score

precision, recall, _ = precision_recall_curve(y_true, probs_softmax[:, 1])
avg_precision = average_precision_score(y_true, probs_softmax[:, 1])

plt.plot(recall, precision, label=f"AP = {avg_precision:.4f}")
plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Precision-Recall Curve")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import os
import pandas as pd
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback
)
import numpy as np
import datasets.formatting.formatting
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import StratifiedShuffleSplit
import torch.nn as nn
from torch.nn import functional as F
import matplotlib.pyplot as plt
import seaborn as sns

# -----------------------------------------------------
# ✅ Monkey patch for NumPy 2.0 error in datasets
def fixed_arrow_array_to_numpy(self, pa_array):
    array = pa_array.to_pandas().values
    return np.array(array, copy=True)

datasets.formatting.formatting.NumpyArrowExtractor._arrow_array_to_numpy = fixed_arrow_array_to_numpy
# -----------------------------------------------------

# ✅ Disable wandb
os.environ["WANDB_DISABLED"] = "true"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ✅ Load dataframe with better preprocessing
df = pd.read_excel(r"BanglaBlendCleanedDataWithEnglishTranslation.xlsx")
df = df.dropna()


# ✅ Keep only needed columns and rename
df = df[["Sentence", "Labels"]].copy()
df = df.rename(columns={"Sentence": "text", "Labels": "label"})
df["label"] = df["label"].astype(int)

# ✅ Basic text preprocessing
def preprocess_text(text):
    """Basic text preprocessing"""
    if pd.isna(text):
        return ""
    text = str(text).strip()
    # Remove extra whitespace
    text = ' '.join(text.split())
    return text

df["text"] = df["text"].apply(preprocess_text)

# ✅ Remove empty texts and duplicates
df = df[df["text"].str.len() > 0]
df = df.drop_duplicates(subset=["text"])

print("Columns after clean & rename:", df.columns)
print("Data shape:", df.shape)
print("Label distribution:")
print(df["label"].value_counts())
print(df.head())

# ✅ Load tokenizer and model with built-in regularization
model_name = "sagorsarker/bangla-bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Use standard model with built-in dropout configuration
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, 
    num_labels=2,
    hidden_dropout_prob=0.5,  # Increase dropout for regularization
    attention_probs_dropout_prob=0.3,  # Attention dropout
    classifier_dropout=0.1  # Classifier dropout
).to(device)

# ✅ Advanced tokenization with dynamic padding
def tokenize(batch):
    return tokenizer(
        batch["text"], 
        padding=False,  # Dynamic padding in data collator
        truncation=True, 
        max_length=256,  # Increased max length for better context
        return_tensors=None
    )

# ✅ Create dataset with stratified split
dataset = Dataset.from_pandas(df)
dataset = dataset.map(tokenize, batched=True, remove_columns=["text"])

# ✅ Stratified train-validation-test split
def create_stratified_splits(dataset, train_size=0.7, val_size=0.15, test_size=0.15, random_state=42):
    labels = [item['label'] for item in dataset]
    
    # First split: train and temp (val + test)
    sss1 = StratifiedShuffleSplit(n_splits=1, test_size=(val_size + test_size), random_state=random_state)
    train_idx, temp_idx = next(sss1.split(range(len(labels)), labels))
    
    # Second split: val and test from temp
    temp_labels = [labels[i] for i in temp_idx]
    val_ratio = val_size / (val_size + test_size)
    sss2 = StratifiedShuffleSplit(n_splits=1, test_size=(1 - val_ratio), random_state=random_state)
    val_idx_temp, test_idx_temp = next(sss2.split(range(len(temp_idx)), temp_labels))
    
    val_idx = [temp_idx[i] for i in val_idx_temp]
    test_idx = [temp_idx[i] for i in test_idx_temp]
    
    return {
        'train': dataset.select(train_idx),
        'validation': dataset.select(val_idx),
        'test': dataset.select(test_idx)
    }

dataset_splits = create_stratified_splits(dataset)

# ✅ Set torch format
for split in dataset_splits:
    dataset_splits[split].set_format(type="torch", columns=["input_ids", "attention_mask", "label"])

print(f"Train size: {len(dataset_splits['train'])}")
print(f"Validation size: {len(dataset_splits['validation'])}")
print(f"Test size: {len(dataset_splits['test'])}")

# ✅ Data collator for dynamic padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# ✅ Enhanced metrics function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }



# ✅ Enhanced training arguments with regularization
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=20,  # Reduced epochs to prevent overfitting
    per_device_train_batch_size=16,  # Increased batch size
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,  # Effective batch size = 32
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    
    # ✅ Learning rate scheduling
    learning_rate=3e-5,  # Slightly higher learning rate
    lr_scheduler_type="cosine",
    warmup_steps=500,
    
    # ✅ Regularization techniques
    weight_decay=0.1,  # L2 regularization
    adam_epsilon=1e-6,
    max_grad_norm=1.0,  # Gradient clipping
    label_smoothing_factor=0.1,  # Built-in label smoothing
    
    # ✅ Early stopping and model selection
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    
    # ✅ Evaluation and saving
    eval_steps=100,
    save_steps=200,
    save_total_limit=3,
    
    # ✅ Other optimizations
    dataloader_num_workers=2,
    remove_unused_columns=False,
    report_to=[]
)

# ✅ Custom Trainer to track training history
class CustomTrainer(Trainer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.train_losses = []
        self.val_losses = []
        self.val_accuracies = []
        self.epochs = []
    
    def log(self, logs):
        super().log(logs)
        # Track training loss
        if "train_loss" in logs:
            self.train_losses.append(logs["train_loss"])
        
        # Track validation metrics
        if "eval_loss" in logs:
            self.val_losses.append(logs["eval_loss"])
            self.val_accuracies.append(logs.get("eval_accuracy", 0))
            self.epochs.append(len(self.val_losses))

# ✅ Standard trainer with built-in label smoothing
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset_splits["train"],
    eval_dataset=dataset_splits["validation"],
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

# ✅ Train with mixed precision for efficiency
print("🚀 Starting training...")
trainer.train()

# ✅ Evaluate on validation set
print("\n📊 Validation Results:")
val_metrics = trainer.evaluate()
for key, value in val_metrics.items():
    print(f"{key}: {value:.4f}")

# ✅ Evaluate on test set
print("\n📊 Test Results:")
test_metrics = trainer.evaluate(dataset_splits["test"])
for key, value in test_metrics.items():
    print(f"test_{key}: {value:.4f}")

# ✅ Calculate overfitting metrics
train_metrics = trainer.evaluate(dataset_splits["train"])
overfitting_accuracy = train_metrics['eval_accuracy'] - test_metrics['eval_accuracy']
overfitting_f1 = train_metrics['eval_f1'] - test_metrics['eval_f1']

print(f"\n🎯 Overfitting Analysis:")
print(f"Training Accuracy: {train_metrics['eval_accuracy']:.4f}")
print(f"Test Accuracy: {test_metrics['eval_accuracy']:.4f}")
print(f"Accuracy Gap (overfitting): {overfitting_accuracy:.4f}")
print(f"F1 Gap (overfitting): {overfitting_f1:.4f}")

# ✅ Save the best model
model.save_pretrained("./final_bangla_bert_model")
tokenizer.save_pretrained("./final_bangla_bert_model")
print("\n✅ Best model saved at ./final_bangla_bert_model")

# ✅ Generate predictions for confusion matrix
def get_predictions(trainer, dataset):
    """Get predictions from the model"""
    predictions = trainer.predict(dataset)
    y_pred = predictions.predictions.argmax(-1)
    y_true = predictions.label_ids
    return y_true, y_pred

# ✅ Create confusion matrix
def plot_confusion_matrix(y_true, y_pred, class_names=None, title="Confusion Matrix"):
    """Plot confusion matrix"""
    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.show()
    
    return cm

# ✅ Plot training history
def plot_training_history(trainer):
    """Plot training and validation loss/accuracy over epochs"""
    epochs = range(1, len(trainer.val_losses) + 1)
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot losses
    ax1.plot(epochs, trainer.val_losses, 'b-', label='Validation Loss', linewidth=2)
    if trainer.train_losses:
        # Match training losses to validation epochs
        train_epochs = np.linspace(1, len(trainer.val_losses), len(trainer.train_losses))
        ax1.plot(train_epochs, trainer.train_losses, 'r-', label='Training Loss', linewidth=2)
    
    ax1.set_title('Training & Validation Loss')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Loss')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot validation accuracy
    ax2.plot(epochs, trainer.val_accuracies, 'g-', label='Validation Accuracy', linewidth=2)
    ax2.set_title('Validation Accuracy')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# ✅ Generate confusion matrices for validation and test sets
print("\n📊 Generating Confusion Matrices...")

# Get unique class names
class_names = [f"Class {i}" for i in sorted(df['label'].unique())]

# Validation set confusion matrix
val_true, val_pred = get_predictions(trainer, dataset_splits["validation"])
print(f"\nValidation Confusion Matrix:")
val_cm = plot_confusion_matrix(val_true, val_pred, class_names, "Validation Set Confusion Matrix")
print(val_cm)

# Test set confusion matrix
test_true, test_pred = get_predictions(trainer, dataset_splits["test"])
print(f"\nTest Confusion Matrix:")
test_cm = plot_confusion_matrix(test_true, test_pred, class_names, "Test Set Confusion Matrix")
print(test_cm)

# ✅ Plot training history
print("\n📈 Training History:")
plot_training_history(trainer)

# ✅ Additional classification metrics
from sklearn.metrics import classification_report

print("\n📊 Detailed Classification Report (Test Set):")
print(classification_report(test_true, test_pred, target_names=class_names))

print("\n📊 Detailed Classification Report (Validation Set):")
print(classification_report(val_true, val_pred, target_names=class_names))

# ✅ Optional: Model inference example
def predict_text(text, model, tokenizer, device):
    """Function to predict on new text"""
    model.eval()
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=256
    ).to(device)
    
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
        predicted_class = torch.argmax(predictions, dim=-1)
    
    return predicted_class.item(), predictions.cpu().numpy()

# ✅ Example usage
sample_text = "এটি একটি পরীক্ষা বাক্য।"
try:
    pred_class, pred_probs = predict_text(sample_text, model, tokenizer, device)
    print(f"\nSample prediction:")
    print(f"Text: {sample_text}")
    print(f"Predicted class: {pred_class}")
    print(f"Probabilities: {pred_probs}")
except Exception as e:
    print(f"Prediction error: {e}")